Imports

In [ ]:
import preamble

import random
import os
import time
from robot import Robot
from console_command_thread import ConsoleCommandThread
from phrase_trial_data import PhraseTrialData
from vocalizer import Vocalizer
from stoppable_thread import StoppableThread
from virtual_dynamics import SimpleVirtualDynamics
from timer import Timer
import numpy as np

Bake corpus (which is a subset of the universe of phrases)

In [ ]:
universe = {
    'prefix': [
        'I need you to',
        'I am telling you to',
        'I think you should',
        'It would be nice if you',
        'You should',
        'I think you should',
        'Can you',
        'Could you',
        'Please',
        'You must',
    ],
    'adverb': [
        'slightly',
        'greatly',
        'smoothly',
        'sharply',
        'slowly',
        'quickly',
        'lightly',
        'significantly',
        'softly',
        'harshly',
        'gradually',
        'immediately',
    ],
    'action': {
        'blank': [
            '',
        ],
        'not blank': {
            'without target': [
                'move',
                'shift',
                'adjust',
                'proceed',
                'ease',
                'head',
                'scoot',
                'advance',
                'go',
            ],
            # 'with target': [
            #     'bring it',
            #     'take it',
            #     'move it',
            #     'shift it',
            #     'adjust it',
            #     'proceed it',
            #     'ease it',
            #     'scoot it',
            #     'advance it',
            #     'move yourself',
            #     'shift yourself',
            #     'adjust yourself',
            #     'proceed yourself',
            #     'ease yourself',
            #     'head yourself',
            #     'scoot yourself',
            #     'advance yourself',
            # ],
        }
    },
    'direction': {
        'left': [
            'left',
            'leftward',
            'to the left'
        ],
        'right': [
            'right',
            'rightward',
            'to the right'
        ],
        'up': [
            'up',
            'upward',
            'above',
        ],
        'down': [
            'down',
            'downward',
            'below',
        ],
        'forward': [
            'forward',
            'ahead',
            'in front',
        ],
        'backward': [
            'backward',
            'back',
            'behind',
        ],
    },
    'and': [
        'and',
    ],
}


def choose_random_part(universe, phrase_part_type):
    o = universe[phrase_part_type]
    if isinstance(o, dict):
        return choose_random_part(o, random.choice(list(o.keys())))
    elif isinstance(o, list):
        return random.choice(o)


phrase_categories = [
    ('direction',),
    ('adverb', 'action', 'direction'),
    ('action', 'direction', 'and', 'direction'),
    ('adverb', 'action', 'direction', 'and', 'direction'),
]


def not_dumb_composite_direction(phrase_parts):
    f = {}
    for phrase_part in phrase_parts:
        if phrase_part not in f:
            f[phrase_part] = 0
        f[phrase_part] = f[phrase_part] + 1
    for i in range(0, len(universe['direction']), 2):
        t = 0
        # for j in range(len(universe['direction'][i])):
        #     t += f[universe['direction'][i][j]] if universe['direction'][i][j] in f else 0
        # for j in range(len(universe['direction'][i])):
        #     t += f[universe['direction'][i+1][j]] if universe['direction'][i+1][j] in f else 0
        for axes_dir_words in [
            'left',
            'right',
            'up',
            'down',
            'forward',
            'backward',
        ][i:i+2]:
            for specific_word in universe['direction'][axes_dir_words]:
                t += f[specific_word] if specific_word in f else 0
        if t > 1:
            return False
    return True


NUMBER_OF_USERS = 12
NUMBER_OF_EACH_CATEGORY = 11

# ex dir name: f'corpus__{unique_timestamp}'
CUSTOM_CORPUS = 'corpus__1731966963.793035'

if CUSTOM_CORPUS is None:
    corpus = [universe['direction']]

    for phrase_category in phrase_categories[1:]:
        phrases_of_category = []
        for _ in range(NUMBER_OF_EACH_CATEGORY * NUMBER_OF_USERS):
            phrase_parts = None
            while True:
                phrase_parts = [choose_random_part(
                    universe, phrase_part_type) for phrase_part_type in phrase_category]
                if not_dumb_composite_direction(phrase_parts):
                    break
            phrase_parts = [
                phrase_part for phrase_part in phrase_parts if phrase_part != '']
            phrases_of_category.append(' '.join(phrase_parts))
        corpus.append(phrases_of_category)

    output_dir = f"corpus__{time.time()}"
    os.makedirs(output_dir, exist_ok=True)

    for i, phrases_of_category in enumerate(corpus):
        with open(os.path.join(output_dir, f"{i}__{'_'.join(phrase_categories[i])}.txt"), "w") as f:
            f.write("\n".join(phrases_of_category))

    for i in range(NUMBER_OF_USERS):
        user_phrases = []

        for category in corpus[1:]:
            for j in range(NUMBER_OF_EACH_CATEGORY):
                user_phrases.append(category[(i*NUMBER_OF_EACH_CATEGORY)+j])

        random.shuffle(user_phrases)

        user_phrases = [
            'left',
            'right',
            'up',
            'down',
            'forward',
            'backward',
        ] + user_phrases

        with open(os.path.join(output_dir, f"user_{i}_phrases.txt"), "w") as f:
            f.write("\n".join(user_phrases))
else:
    corpus = []
    files = sorted(
        [f for f in os.listdir(CUSTOM_CORPUS) if f.endswith(
            ".txt") and 'user' not in f],
        key=lambda x: int(x.split('__')[0])
    )
    for filename in files:
        with open(os.path.join(CUSTOM_CORPUS, filename), "r") as file:
            phrases = file.read().splitlines()
            corpus.append(phrases)

    output_dir = CUSTOM_CORPUS

Experiment

In [ ]:
USER_ID = 0

user_phrases = None

with open(os.path.join(output_dir, f"user_{USER_ID}_phrases.txt"), "r") as f:
    user_phrases = f.read().splitlines()

for p in user_phrases:
    print(p)

NUM_OF_STAGES = 6
stage = 0

def next_stage():
    global stage
    stage = (stage + 1) % NUM_OF_STAGES

current_phrase_index = 0

with Robot(
    '169.254.9.43',
    translational_force_deadband=6.0,
    rotational_force_deadband=0.5
) as r, ConsoleCommandThread(['', 'restart']) as c:
    current_phrase_trial_data = None
    save_current_phrase_trial = False

    vocalizer = Vocalizer(0)

    AXES = Robot.TRANSLATION

    init_point = r.get_pose(AXES)

    timer = Timer()

    vd = SimpleVirtualDynamics(
        M=10,
        B=25,
        K=0,
    )

    while c.is_alive():
        period_start = r.control.initPeriod()

        if stage == 0: # wait for start signal + keep robot still
            if c.poll_command(''):
                next_stage()
                c.reset()
            elif c.poll_command('exit'):
                break
            r.set_velocity(Robot.get_axes(Robot.zeroed_translation_rotation(), AXES), AXES, acceleration=10)
        elif stage == 1: # initiate phrase trial + utter phrase
            r.set_velocity(Robot.get_axes(Robot.zeroed_translation_rotation(), AXES), AXES, acceleration=10)
            current_phrase = user_phrases[current_phrase_index]
            print(current_phrase)
            current_phrase_trial_data = PhraseTrialData(current_phrase, USER_ID)
            vocalizer.utter(current_phrase, True)
            timer.reset()
            next_stage()
            c.reset()
        elif stage == 2: # robot can move + phrase trial data recording + check for signal
            time = timer.t()

            dt = timer.dt()

            position = r.get_pose(AXES)

            velocity = r.get_velocity(AXES)

            force = r.get_force(AXES)

            current_phrase_trial_data.append(
                time,
                dt,
                position,
                velocity,
                force,
            )

            vd.apply_force(force, dt)

            r.set_velocity(vd.get_velocity(), AXES, acceleration=10)

            if c.poll_command(''):
                save_current_phrase_trial = True
                next_stage()
                c.reset()
            elif c.poll_command('restart'):
                current_phrase_index -= 1
                next_stage()
                c.reset()
            elif c.poll_command('exit'):
                break
        elif stage == 3:
            vd.v = 0.0
            r.set_velocity(Robot.get_axes(Robot.zeroed_translation_rotation(), AXES), AXES, acceleration=10)
            if save_current_phrase_trial:
                current_phrase_trial_data.save(current_phrase_index)
                save_current_phrase_trial = False
            current_phrase_index += 1
            next_stage()
            c.reset()
        elif stage == 4:
            position = r.get_pose(AXES)
            position_delta = init_point - position
            position_delta_magnitude = np.linalg.norm(position_delta)
            position_delta_normalized = position_delta / position_delta_magnitude if position_delta_magnitude != 0 else 0
            return_velocity = 0.2
            r.set_velocity(return_velocity*position_delta_normalized, AXES, acceleration=2)
            if position_delta_magnitude < 0.001:
                r.set_velocity(Robot.get_axes(Robot.zeroed_translation_rotation(), AXES), AXES, acceleration=10)
                next_stage()
        elif stage == 5:
            r.set_velocity(Robot.get_axes(Robot.zeroed_translation_rotation(), AXES), AXES, acceleration=10)
            c.reset()
            next_stage()

        r.control.waitPeriod(period_start)

print(current_phrase_index)